# simple_model

Minimal notebook to ask for cycling routes by natural language and fetch matching rows from the engineered routes DataFrame using Gemini tool-calling.

In [1]:
import os
import pandas as pd
from google import genai
from google.genai import types

In [2]:
# Load API key and data
GKEY = os.getenv("GEMINIKEY")
assert GKEY, "Missing GEMINIKEY in environment/.env"

df = pd.read_csv("/Users/eugeneleach/code/Eugle3/cycle_more/Notebooks/UK_Engineered_Data.csv")
print(f"Loaded {len(df)} routes")


Loaded 7717 routes


In [3]:
def get_routes(min_distance: float, min_duration: float):
    """Return routes with distance >= min_distance and duration >= min_duration."""
    return df.loc[(df["distance_m"] >= min_distance) & (df["duration_s"] >= min_duration)]


In [4]:
# Tool schema Gemini will use to call Python
get_routes_tool = {
    "name": "get_routes",
    "description": "Return routes that meet minimum distance and duration thresholds.",
    "parameters": {
        "type": "object",
        "properties": {
            "min_distance": {
                "type": "number",
                "description": "Minimum distance in metres (e.g. 5000)",
            },
            "min_duration": {
                "type": "number",
                "description": "Minimum duration in seconds (e.g. 3600)",
            },
        },
        "required": ["min_distance", "min_duration"],
    },
}

filter_tools = types.Tool(function_declarations=[get_routes_tool])


In [5]:
# Gemini client
client = genai.Client(api_key=GKEY)
MODEL = "gemini-2.0-flash"


In [6]:
def run_query(user_question: str, head: int = 5):
    """Ask Gemini to pick thresholds, then return matching routes."""
    resp = client.models.generate_content(
        model=MODEL,
        contents=user_question,
        config=types.GenerateContentConfig(tools=[filter_tools]),
    )
    fc = resp.candidates[0].content.parts[0].function_call
    args = fc.args
    routes = get_routes(**args)
    print(f"Gemini called {fc.name} with {args}")
    return routes.head(head)


In [7]:
# Example
user_question = "Find rides at least 50000 meters and at least 3600 seconds"
run_query(user_question)

Gemini called get_routes with {'min_distance': 50000, 'min_duration': 3600}


,Unnamed: 0.1,Unnamed: 0,id,name,distance_m,duration_s,ascent_m,descent_m,steps,turns,...,flat (0%),downhill_gentle (-5% to 0%),uphill_steep (5% to 7%),uphill_extreme (>10%),downhill_extreme (<-15%),downhill_moderate (-7% to -5%),downhill_steep (-10% to -7%),downhill_very_steep (-15% to -10%),Average_Speed,Turn_Density
1295,1295,1345,162996,Unnamed route,57446.2,11685.8,1065.5,1024.5,20,13,...,22.37,24.68,4.25,0.00,0.00,9.15,0.83,0.00,4.915898,0.226299
1302,1302,1352,918684,Unnamed route,55466.7,12915.9,1126.9,1110.9,23,13,...,30.43,33.54,0.71,0.18,0.09,5.41,3.64,0.62,4.294451,0.234375
1305,1305,1355,2756970,Coastal Route,58638.6,11732.8,693.5,676.5,16,14,...,56.87,19.58,0.49,0.00,0.40,0.69,0.00,0.00,4.997835,0.238751
1548,1548,1598,15979000,Unnamed route,54585.7,11048.4,144.8,154.8,29,20,...,100.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,4.940598,0.366396
1578,1578,1628,51479,NCN National Route 754,93356.7,18943.8,797.5,725.5,32,21,...,94.34,1.29,0.38,0.00,0.00,0.00,0.00,0.00,4.928087,0.224944


## LLM VERSION 2

In [8]:
def change_route_distance(route_id, multiplier: float):
    """Return a copy of the route with distance multiplied by `multiplier`.

    Casts IDs to string to be robust to int/str input.
    Does not modify the original DataFrame.
    """
    rid = str(route_id)
    route = df.loc[df["id"].astype(str) == rid]
    if route.empty:
        print(f"No route found with id {route_id}")
        return route
    return route.assign(distance_m=route["distance_m"] * float(multiplier))


In [9]:
# Tool schema for changing a single route's distance
change_route_distance_tool = {
    "name": "change_route_distance",
    "description": "Return the route with its distance multiplied by the given factor (other features unchanged).",
    "parameters": {
        "type": "object",
        "properties": {
            "route_id": {
                "type": "string",
                "description": "The ID of the route to change.",
            },
            "multiplier": {
                "type": "number",
                "description": "Factor to multiply the route's distance by.",
            },
        },
        "required": ["route_id", "multiplier"],
    },
}

distance_tools = types.Tool(function_declarations=[change_route_distance_tool])


In [10]:
def run_distance_change_query(user_question):
    """Ask Gemini to pick a route id and multiplier, then return the modified route copy."""
    resp = client.models.generate_content(
        model=MODEL,
        contents=user_question,
        config=types.GenerateContentConfig(tools=[distance_tools]),
    )
    fc = resp.candidates[0].content.parts[0].function_call
    args = fc.args
    route = change_route_distance(**args)
    print(f"Gemini called {fc.name} with {args}")
    return route


In [11]:
# Example
user_question = "For route id 162996, multiply its distance by 2"
run_distance_change_query(user_question)


Gemini called change_route_distance with {'route_id': '162996', 'multiplier': 2}


,Unnamed: 0.1,Unnamed: 0,id,name,distance_m,duration_s,ascent_m,descent_m,steps,turns,...,flat (0%),downhill_gentle (-5% to 0%),uphill_steep (5% to 7%),uphill_extreme (>10%),downhill_extreme (<-15%),downhill_moderate (-7% to -5%),downhill_steep (-10% to -7%),downhill_very_steep (-15% to -10%),Average_Speed,Turn_Density
1295,1295,1345,162996,Unnamed route,114892.4,11685.8,1065.5,1024.5,20,13,...,22.37,24.68,4.25,0.0,0.0,9.15,0.83,0.0,4.915898,0.226299
